In [22]:
from transformers import pipeline
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os

#print(torch.cuda.device_count())
#os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

#モデル読み込み
model_name = "meta-llama/Llama-2-7b-chat-hf"

model_name = "tokyotech-llm/Swallow-MS-7b-v0.1"
model_name = "cyberagent/open-calm-7b"

model_name="Local-Novel-LLM-project/Vecteus-v1"
#model_name="nvidia/Llama3-ChatQA-1.5-8B"

model_name="../model/------llm-models-hf-2ja1_part_inst_all_lr_1e-5"
model_name="../model/------llm-models-hf-2ja1_part_inst_all_lr_1e-5"
model_name="../X_merge/merged_models/0507test"
#model_name="../X_merge/merged_models/mergoo_llama_test"

model_path="../model/------llm-models-hf-2ja1_part_inst_all-parquet_lr_1e-5"
model_path="../../llm/models/hf/step41300",
tokenizer = AutoTokenizer.from_pretrained(model_name)


/home/hatakeyama/miniconda3/envs/llmeval2/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [23]:
t_list=[
"\n\n### 応答:\n",
"\n\n### 応答:\n「",
"\n\n### 応答:\nああ",
"\n\n### 応答:\naaa",
"1\n\n### 応答:\naaa",
"あ\n\n### 応答:\naaa",
"」\n\n### 応答:\naaa",
]
t_list=[
"\n\n### 指示:\n",
"\n\n### 指示:\n「",
"\n\n### 指示:\nああ",
"\n\n### 指示:\naaa",
"1\n\n### 指示:\naaa",
"あ\n\n### 指示:\naaa",
"」\n\n### 指示:\naaa",
]
for t in t_list:
    print(tokenizer.encode(t),tokenizer.tokenize(t))


[5092, 272, 3994, 327, 2] ['▁###', '▁', '指示', ':']
[5092, 272, 3994, 327, 337, 2] ['▁###', '▁', '指示', ':', '▁「']
[5092, 272, 3994, 327, 8722, 2] ['▁###', '▁', '指示', ':', '▁ああ']
[5092, 272, 3994, 327, 272, 55841, 2] ['▁###', '▁', '指示', ':', '▁', 'aaa']
[272, 279, 5092, 272, 3994, 327, 272, 55841, 2] ['▁', '1', '▁###', '▁', '指示', ':', '▁', 'aaa']
[4543, 5092, 272, 3994, 327, 272, 55841, 2] ['▁あ', '▁###', '▁', '指示', ':', '▁', 'aaa']
[272, 285, 5092, 272, 3994, 327, 272, 55841, 2] ['▁', '」', '▁###', '▁', '指示', ':', '▁', 'aaa']


In [24]:
tokenizer.encode(t)

[272, 285, 5092, 272, 3994, 327, 272, 55841, 2]

In [25]:
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            )


Loading checkpoint shards: 100%|██████████| 14/14 [00:33<00:00,  2.42s/it]


In [26]:
#model.load_adapter(model_path)

In [27]:
#層の表示
for name, param in model.named_parameters():
    print(name)
    if "gate" in name:
        print("***",name)

model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.block_sparse_moe.gate.weight
*** model.layers.0.block_sparse_moe.gate.weight
model.layers.0.block_sparse_moe.experts.0.w1.weight
model.layers.0.block_sparse_moe.experts.0.w2.weight
model.layers.0.block_sparse_moe.experts.0.w3.weight
model.layers.0.block_sparse_moe.experts.1.w1.weight
model.layers.0.block_sparse_moe.experts.1.w2.weight
model.layers.0.block_sparse_moe.experts.1.w3.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.v_proj.weight
model.layers.1.self_attn.o_proj.weight
model.layers.1.block_sparse_moe.gate.weight
*** model.layers.1.block_sparse_moe.gate.weight
model.layers.1.block_sparse_moe.experts.0.w1.weight
model.layers.1.block_sparse_m

In [28]:
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()



In [38]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=50, 
              repetition_penalty=1.2,
              temperature=0.6,
              #repetition_penalty=1.,
              #temperature=0.1,
              #top_p=1.0,
              #top_k=0.
              )


In [39]:
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n\n### 応答:\n"

def gen_prompt(q):
    return f"{question_template}{q}{answer_template}"
    return question_template+q+answer_template

In [42]:
questions=[
"こんにちは",
"元気ですか?",
"いい天気ですね",
"明日の天気は?",
"計算してください 1+1はいくつですか｡",
"フィボナッチ数列を生成するpythonのコードを作成してください",
"ドラえもんの友達はだれですか?",
"レイリー散乱とはなんですか",
"hello!",
"今の天気は晴れです｡この結果をjsonで出力してください｡ ",
"純粋理性批判はたぬきに理解できますか?",
]

for question in questions:
    #question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    #inp=f"{question_template}{question}<SEP>応答<SEP>"
    inp=gen_prompt(question)
    inp=f"質問: {question}回答: "
    print("--------------------")
    print(question)
    print(pipe(inp)[0]["generated_text"][len(inp):])

--------------------
こんにちは
 私は最近、日本の文化や伝統を中心に多くの人々が参加しています。私たちはこのような文化や歴史について非常に興味がありますし、また他にも様々な文化や歴史に関する情報も提供されてい
--------------------
元気ですか?
 最近、私の周りでは多くの人が集まるようになりました。特に、私は毎日仕事をしていますし、忙しい時間も過ごしているためです。しかし、今回の質問については、皆様のご意見を
--------------------
いい天気ですね
 10月の晴れです。 11月の風は、最高です。 12月の雨は、最高です。 13月の日は、最高です。 14月の太陽は、最高
--------------------
明日の天気は?
明日の天候は雨です。今日の天気が良いと思いますが、雨が降る可能性があります。
--------------------
計算してください 1+1はいくつですか｡
 20です。数字の合計は20です。これは、20×2=4で計算します。つまり、20×3=6です。このように計算が行われます。また、20×4
--------------------
フィボナッチ数列を生成するpythonのコードを作成してください
 Python Python
--------------------
ドラえもんの友達はだれですか?
 ドラえもんの友人として、私が好きな名前を挙げさせていただきます。その理由について教えてください。
--------------------
レイリー散乱とはなんですか
私の友人が、彼らを愛して、彼らに愛した。彼らは、彼らの心を持って、彼らの心を守る。」という言葉で語られます。この文は、「
--------------------
hello!
 1. I'm a student in the field of computer science and have been doing research on various topics related to this area for several years. However, there are some areas that need improvement or more attention from me.
2.
---

# ベンチマーク問題での応答評価

In [47]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=50, 
              repetition_penalty=1.,
              temperature=0.1,
              top_p=1.0,
              top_k=0.
              )

In [48]:
#ベンチマーク問題への応答性の軽い評価
import pandas as pd
jaster_path="bench_data/jaster.txt"
with open(jaster_path,"r") as f:
    input_list=f.readlines()

for question in input_list:
    #question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    #inp=f"{question_template}{question}<SEP>応答<SEP>"
    inp=gen_prompt(question)
    inp=f"質問: {question}回答: "
    print("--------------------")
    print(question)
    print(pipe(inp)[0]["generated_text"][len(inp):])

--------------------
前提と仮説の関係をentailment、non-entailmentの中から回答してください。それ以外には何も含めないことを厳守してください。\n\n制約：\n- 前提に対して仮説が同じ意味を含む場合は、entailmentと出力\n- 前提に対して仮説が異なる意味を含む場合は、non-entailmentと出力,前提：カップルが学生を追い回した\n仮説：学生をカップルが追い回した,前提と仮説の関係をentailment、non-entailmentの中から回答してください。それ以外には何も含めないことを厳守してください。\n\n制約：\n- 前提に対して仮説が同じ意味を含む場合は、entailmentと出力\n- 前提に対して仮説が異なる意味を含む場合は、non-entailmentと出力\n前提：カップルが学生を追い回した\n仮説：学生をカップルが追い回した



/home/hatakeyama/miniconda3/envs/llmeval2/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/hatakeyama/miniconda3/envs/llmeval2/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


 前提と仮説の関係をentailment、non-entailmentの中から回答: \n\n\n\n\n\n\n\n\n\n\n\n\n\n
--------------------
前提と仮説の関係をentailment、contradiction、neutralの中から回答してください。それ以外には何も含めないことを厳守してください。\n\n制約：\n- 前提から仮説が、時間関係上導出可能である場合はentailmentと出力\n- 前提と仮説が両立しえない場合はcontradictionと出力\n- そのいずれでもない場合はneutralと出力,前提：ボブは選挙に撤回を表明している。\n仮説：ボブは現在選挙に撤回を表明している。,前提と仮説の関係をentailment、contradiction、neutralの中から回答してください。それ以外には何も含めないことを厳守してください。\n\n制約：\n- 前提から仮説が、時間関係上導出可能である場合はentailmentと出力\n- 前提と仮説が両立しえない場合はcontradictionと出力\n- そのいずれでもない場合はneutralと出力\n前提：ボブは選挙に撤回を表明している。\n仮説：ボブは現在選挙に撤回を表明している。



KeyboardInterrupt: 

In [46]:
jmt_path="bench_data/jmt.csv"
df=pd.read_csv(jmt_path)
input_list=df["input"].to_list()
for question in input_list:
    #question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    #inp=f"{question_template}{question}<SEP>応答<SEP>"
    inp=gen_prompt(question)
    inp=f"質問: {question}回答: "
    print("--------------------")
    print(question)
    print(pipe(inp)[0]["generated_text"][len(inp):])

--------------------
ディレクトリ内の全てのテキストファイルを読み込み、出現回数が最も多い上位5単語を返すPythonプログラムを開発してください。


/home/hatakeyama/miniconda3/envs/llmeval2/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 ファイル内の各部の文字はすべて出現に依る必要があります。そのため、他の部の文字と比べて出現回数が少ない場合には、次のように Python方法を使ってください。 Python文
--------------------
それを並列化（parallelize）することは可能ですか？
 これは、同様の場合を用いて行われます。同様に同等の場で行われることは、同一の場が同等の場であることを意味します。同等と同等の場では
--------------------
再帰（recursion）を用いてn番目のフィボナッチ数（Fibonacci number）を求めるC++プログラムを書いてください。
C PythonC code C n番のマクロ(F)は、nのFとnのFがそれぞれnにあります。 nのFはnのFでnのFのFのFのFのFのFのF
--------------------
今度は、各数が前の3つの数の和である数列を考えます。最初の3つの数は0、-1、-1です。n番目の数を求めるプログラムを書いてください。
次の3つの数は0、-1、-1です。 n名の数は2です。 n名の数は2です。 n名の数は2です。 n名の数は2です。 n名の
--------------------
HTMLでシンプルなウェブサイトを作成してください。ユーザーがボタンをクリックすると、4つのジョークからランダムに一つ表示されます。
次の文は、" HTML"と "機能"の両方の要素を使用して、"機能"の部分"を"機能"の形"として"機能"の"」を""機能""の""
--------------------
CSSを使ってジョークの色を赤にする方法は何ですか？
 CSSが赤色であることを示して、赤色の赤色と赤色の黄色色を同じようにした場合です。これは CSSの赤色の赤色と赤色の緑色の赤色の赤色の
--------------------
以下に、二つの入力文字列の最長共通部分列（longest common subsequence）の長さを求めるPython関数があります。この関数には何かバグがありますか？

```
def longest_common_subsequence_length(str1, str2):
m = len(str1)
n = len(str2)

dp = [[0] * (n 

KeyboardInterrupt: 